# Using GPT-4 turbo for economic policy stance classification

In this notebook, we take data collected by Benoit et al. ([2016]()) to illustrate how to use GPT-4-turbo through the OpenAI chat completions API to classify texts.

In [1]:
import os
from openai import OpenAI
import tiktoken

import pandas as pd
import json

client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

/var/folders/_c/cm0nk6y92rz2l6ct3npgw9tr0000gn/T/ipykernel_3752/1905943439.py:5: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
MODEL = 'gpt-4-turbo-preview'

In [3]:
from typing import Union, List

class TokenCounter:
    def __init__(self, encoding_name: Union[str, None] = None, model: Union[str, None] = None):
        """
        Initialize the tokenizer with either a model or an encoding name.

        Args:
            encoding_name (Union[str, None]): The name of the encoding to use. Default is None.
            model (Union[str, None]): The model to use for encoding. Default is None.

        Raises:
            ValueError: If neither model nor encoding_name is provided.
            ValueError: If both model and encoding_name are provided.
        """
        # ensure that either model or encoding_name is provided
        if model is None and encoding_name is None:
            raise ValueError("Either `model` or `encoding_name` must be provided.")
        if model is not None and encoding_name is not None:
            raise ValueError("Only one of `model` or `encoding_name` can be provided.")
        if encoding_name:
            self.encoding = tiktoken.get_encoding(encoding_name)
        else:
            self.encoding = tiktoken.encoding_for_model(model)
    
    def count_tokens(self, input: Union[str, List[str]]) -> Union[int, List[int]]:
        """
        Count the number of tokens in the input.

        Args:
            input (Union[str, List[str]]): The input to tokenize. Can be a string or a list of strings.

        Returns:
            Union[int, List[int]]: The number of tokens in the input. If the input is a list, returns a list of token counts.
        """
        if isinstance(input, str):
            return len(self.encoding.encode(input))
        else:
            toks = self.encoding.encode_batch(input)
            return [len(t) for t in toks]

    def __call__(self, input: Union[str, List[str]]) -> Union[int, List[int]]:
        """
        Call the tokenizer on the input. This is equivalent to calling count_tokens.

        Args:
            input (Union[str, List[str]]): The input to tokenize. Can be a string or a list of strings.

        Returns:
            Union[int, List[int]]: The number of tokens in the input. If the input is a list, returns a list of token counts.
        """
        return self.count_tokens(input)

In [4]:
token_counter = TokenCounter(model=MODEL)

In [28]:
# adapt instructions from Benoit et al.'s original crowd coding instructions (see data/benoit_crowdsourced_2016/instructions/) 
instructions = """
Your task is to read sentences from political texts about economic policy issues and classify their stance on the issue.

The sentences you will be asked to interpret come from political party manifestos.

First, you will read a short section from a party manifesto. For the focal sentence enclosed in triple quotes, you will then indicate your best judgment of whether the sentence expresses a left or right economic policy stance.

For each focal sentence, choose one of the following categories: "left", "right".

We tell you below about what we mean by "left" and "right".

## What is a "left" economic policy stance?

**"Left" economic policies** tend to favor one or more of the following: 

- High levels of services provided by the government and state benefits, even if this implies high levels of taxation;
- Public investment. Public ownership or control of sections of business and industry;
- Public regulation of private business and economic activity;
- Support for workers/trade unions relative to employers

## What is a "right" economic policy stance?

**"Right" economic policies** tend to favor one or more of the following: 

- Low levels of taxation, even if this implies low levels of levels of services provided by the government and state benefits;
- Private investment. Minimal public ownership or control of business and industry;
- Minimal public regulation of private business and economic activity;
- Support for employers relative to trade unions/workers

"""

In [6]:
token_counter(instructions)/1000*0.01 # dollar cents per request

0.00307

In [7]:
fp = "../../data/benoit_crowdsourced_2016/benoit_crowdsourced_2016_econ_policy_stance.csv"
df = pd.read_csv(fp)
# keep only gold-standard examples
df = df[df.metadata__gold]
len(df)

225

In [18]:
df.label.value_counts()

label
 1    160
-1     65
Name: count, dtype: int64

In [8]:
df.columns

Index(['uid', 'text', 'label', 'metadata__gold', 'metadata__sentence_id',
       'metadata__pre_sentence', 'metadata__post_sentence'],
      dtype='object')

In [9]:
# construct input 
def construct_input(row):
    out = ""
    if isinstance(row['metadata__pre_sentence'], str):
        out += row['metadata__pre_sentence'].strip() + " "
    out += '"""'
    out += row['text'].strip()
    out += '"""'
    if isinstance(row['metadata__post_sentence'], str):
        out += " " + row['metadata__post_sentence'].strip()
    return out

df['input'] = df.apply(construct_input, axis=1).tolist()

In [29]:
i = 101 # 
df.label.values[i], df['input'].values[i]

(-1,
 'to strengthen the rights of women at work including equal pay for work of equal value and equal treatment. We will ensure that all public authorities and private contractors are equal opportunity employers and we will promote changes to enable those with domestic responsibilities to secure access to employment. """We would restore maternity grants and give a tax allowance to help with child-care costs.""" We would remove the tax on the use of workplace nurseries and encourage wider provision of child-care facilities. UNEMPLOYMENT. Unemployment at present levels is not the inevitable result of new technology or world recession - Japan has only 2.5% unemployment and US unemployment has fallen by two million since 1983.')

In [30]:
messages = [ 
    {"role": "system", "content": instructions},
    {"role": "user", "content": df['input'].values[4]}
]

response = client.chat.completions.create(
    model=MODEL,
    messages=messages,
    seed=42,
    temperature=0.0,
    # response_format={"type": "json_object"},
)

results = response.choices[0].message.content
results

'right'

In [31]:
def classify_text(text):
    messages = [ 
        {"role": "system", "content": instructions},
        {"role": "user", "content": text}
    ]

    response = client.chat.completions.create(
        model=MODEL,
        messages=messages,
        seed=42,
        temperature=0.0,
        # response_format={"type": "json_object"},
    )

    results = response.choices[0].message.content
    return results

In [32]:
samples = df.groupby('label').sample(25, random_state=42)[["uid", "input", "label"]].reset_index(drop=True)
samples

,uid,input,label
0,30001331,Encourage the establishment and success of co-...,-1
1,50008451,We will raise the basic rate of income tax by ...,-1
2,20002551,The dole queue is three times what it was in 1...,-1
3,30000891,We will increase child benefit by Œ£3 a week f...,-1
4,20004521,The Conservatives' taxation and benefit polici...,-1
5,60002701,The University for Industry will be a public/p...,-1
6,20004651,For poorer pensioners we will introduce an add...,-1
7,20004611,The second phase of our proposals will be a re...,-1
8,60004261,Every modern industrial country has a minimum ...,-1
9,20004831,We do not support this discrimination based on...,-1


In [33]:
# tokens in inputs
n_tokens = samples.input.apply(token_counter.count_tokens).sum()
# add token count for instructions
n_tokens += token_counter(instructions) * len(samples)
# add token count for outputs (multiplied by cost factor for output vs. input)
n_tokens += len(samples) * 3

# comopute cost (see https://openai.com/pricing)
n_tokens/1000*0.01 # dollar cents

0.20284

In [34]:
# classify: apply custom classification function to all inputs
results = samples.input.apply(classify_text)

In [36]:
# evaluate: compute performance metrics
from sklearn.metrics import classification_report

id2label = {-1: "left", 1: "right"}
print(classification_report(samples.label.map(id2label), results.values))

              precision    recall  f1-score   support

        left       1.00      0.96      0.98        25
       right       0.96      1.00      0.98        25

    accuracy                           0.98        50
   macro avg       0.98      0.98      0.98        50
weighted avg       0.98      0.98      0.98        50

